In [56]:
#basics -> imports and apikey
import json
import requests as rq
import pandas as pd
import time

apikey = 'RGAPI-78fcbd4c-5116-4c53-b1b3-4c2d6fd07a35'

### I'm just using an excel table where I have the list of summoner names

In [57]:
list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheet_name = 0,
                      header = 0,
                      index_col = 0,
                      usecols = 0,
                      convert_float = True)

#check what was actually extracted into the table
print (list_summoners.head)

<bound method NDFrame.head of Empty DataFrame
Columns: []
Index: [40424646]>


### I use PyMongo (for storing into MongoDB), but if you don't want to store you can skip anything DB related.

In [58]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [59]:
db = client.lol_stats_db
collection = db.summoner_stats

### Old code, as you can see. API version 2.2 still ;)

In [64]:
#test with i=0
r = rq.get('https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-account/' +str(list_summoners.index[0])+
           '?api_key=' + apikey)
#check response code 
r.status_code

200

In [65]:
(r.status_code == 200)

True

### Insert into database

### Id=0 for testing

In [70]:
#insert into database
###result = db.match_list_2016_5x5.remove({ "_id": str(list_summoners.index[0])})
result = db.summoner_stats.insert_one({ "_id": str(list_summoners.index[0]), "stats": json.loads(r.text)})
print (result.inserted_id)

DuplicateKeyError: E11000 duplicate key error collection: lol_stats_db.summoner_stats index: _id_ dup key: { : "40424646" }

### Retrieve stats (for example)

In [68]:
#let's get all recent matches of summoner
i = 1
while i < len(list_summoners.index):  
    #get all matches for a summoner id
    r = rq.get('https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/' +str(list_summoners.index[i]) + apikey)
    while (r.status_code != 200):
        time.sleep(4)
        r = rq.get('https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/' +str(list_summoners.index[i]) + apikey)
    result = db.summoner_stats.insert_one({ "_id": str(list_summoners.index[i]), "stats": json.loads(r.text)} )
    #insert into stats (contains all stats per summoner in season 16)
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(1)

### Each API response was stored into a MongoDB above; below I just retrieve an individual instance and then, using the JSON structure, I test a few variables.

In [69]:
cursor = db.summoner_matches.find({"_id": str(list_summoners.index[0])})
for document in cursor:
    print (document['stats']['playerStatSummaries'][0]['queue']  == '420')
    
if (document['stats']['playerStatSummaries'][0]['queue']):
    print ("OK")

KeyError: 'playerStatSummaries'

### Let's extract all (wanted) stats per summoner. This is using the JSON from the Mongo Database, but you could feed the API response directly. But, honestly, it is better to have a middle DB; this way you can run the query (it can take long) and do the analysis much later without worrying.

In [42]:
#we want wins from 'RankedFlexSR', 'Unranked'
#wins and losses from 'RankedSolo5x5', 'RankedTeam5x5'
#and total wins per summoner


i=0
summoner_list = []
que = []
gameid = []
lane = []


while (i<len(list_summoners)):
    cursor = db.summoner_stats.find({"_id": str(list_summoners.index[i])})
    for document in cursor:
        summoner_list.append(document['_id'])
        queue = 0
        gameid = 0
        lane = 0
        j=0
        while (j<len(document['stats']['playerStatSummaries'])):
            queue = document['stats']['playerStatSummaries'][j]['queue']
            if (queue == '420' or queue == 'RankedSolo'):
                gameid =document['stats']['gameId'][j]['gameid']
            elif (queue == 'RankedSolo5x5' or queue == 'RankedTeam5x5'):
                if (document['stats']['playerStatSummaries'][j]['aggregatedStats']):
                    ranked_kills = ranked_kills + document['stats']['playerStatSummaries'][j]['aggregatedStats']['totalChampionKills']
                    ranked_assists = ranked_assists + document['stats']['playerStatSummaries'][j]['aggregatedStats']['totalAssists']
                else:
                    ranked_kills = ranked_kills + 0
                    ranked_assists = ranked_assists + 0
            j=j+1
        total_wins.append(wins)
        total_ranked_wins.append(ranked_wins)
        total_ranked_losses.append(ranked_losses)
        total_ranked_kills.append(ranked_kills)
        total_ranked_assists.append(ranked_assists) 
    i=i+1
        
print (summoner_list)
print (total_wins)
print (total_ranked_assists)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

### Above I only added all the "wanted" variables to five different lists. From these ordered lists (and the ID) I'll build a pandas dataframe easily:

In [ ]:
d = {'SummonerID' : summoner_list,'Queue' : queue, 'MatchID' : gameid,
     'Position' : lane,}
table_summoner_stats = pd.DataFrame(d)

### An then you can save it to any desired format; here, it was excel, but CSV should be even simpler.

In [ ]:
#Now, let's move this to an excel file

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('SummonerStats.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
table_summoner_stats.to_excel(writer, sheet_name='table_summoner_stats')
# Close the Pandas Excel writer and output the Excel file.
writer.save()